In [1]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import io, color
from tifffile import imsave
from tqdm import tqdm
from time import time
from math import ceil
import cv2
import glob
import os
from shutil import copyfile
from random import sample
from PIL import Image
from tqdm import tqdm

def image_show(image, nrows=1, ncols=1, cmap='gray',size = 8):
#     large_image = np.zeros((image.shape[0]+2,image.shape[1]+2), dtype = image.dtype) #add black border
#     large_image[1:-1,1:-1] = image
    
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(size, size))
    ax.imshow(image, cmap='gray')
    ax.axis('off')
    plt.show()
    return fig, ax

In [ ]:
biopsy_group = 's001_o' #change

v2v_folder = 'Z:\\WX\\WX_test_dod\\whobio_sets\\vid2vid_results\\'+biopsy_group+'\\test_latest' #c
seq_list = glob.glob(os.path.join(v2v_folder, 'dod*'))
print(os.path.join(v2v_folder, 'dod*'))
seq_name_list = []
for seq in seq_list:
    seq_name = os.path.basename(seq)[:-1]
#     print(seq_name)
    if seq_name not in seq_name_list:
        seq_name_list.append(seq_name)
print(len(seq_name_list))

p2p_folder = 'Z:\\WX\\WX_test_dod\\whobio_sets\\'+biopsy_group+'\\test\\ch0' #c

full_seq_folder = os.path.join(os.path.dirname(v2v_folder), 'full_seq')
if not os.path.exists(full_seq_folder):
    os.makedirs(full_seq_folder)
        
for seq_name in tqdm(seq_name_list): #seq_name_list
   
    print('-----------', seq_name)
        
    top_seq_dir = os.path.join(v2v_folder, seq_name+'t')
    top_seq_frame_list = glob.glob(os.path.join(top_seq_dir,'fake_B*'))
    nf_top = len(top_seq_frame_list)
#     print(nf_top)
    #read top
    top_half = np.zeros((nf_top,1024,1024),dtype = np.uint8)
    print('reading top half...')
    for idx in range(nf_top): #range(nf_top)
        frame_dir = top_seq_frame_list[idx]
        frame = (io.imread(frame_dir, as_gray = True)*255).astype(np.uint8) 
#         image_show(frame)
        top_half[nf_top-idx-1] = frame #go backwards,grayscale
    print('len(top_half)', len(top_half))
#     image_show(top_half[len(top_half)//2])

    bot_seq_dir = os.path.join(v2v_folder, seq_name+'b')
    bot_seq_frame_list = glob.glob(os.path.join(bot_seq_dir,'fake_B*'))
    nf_bot = len(bot_seq_frame_list)
#     print(nf_bot)
    #read bottom
    print('reading bottom half...')
    bot_half = np.zeros((nf_bot,1024,1024),dtype = np.uint8)
    for idx in range(nf_bot):
        frame_dir = bot_seq_frame_list[idx]
        bot_half[idx] = (io.imread(frame_dir, as_gray = True)*255).astype(np.uint8)
    print('len(bot_half)', len(bot_half))
#     image_show(bot_half[len(bot_half)//2])
    
    #separate use and blend seq from top and botttom
    full_height = nf_top+nf_bot-30 #30 infrence frame overlap
    print('full_height', full_height)
    full_seq = np.zeros((full_height,1024,1024), dtype = np.uint8)
    
    top_use = top_half[:nf_top-20] #now top_half is in top-to-bottom direction, same as botttom_half
    top_blend = top_half[nf_top-20:nf_top-10] #last 20-10 frames are for blending, discard last 10 frames
    
    bot_use = bot_half[20:] #discard first 10 frames, first 10-20 frames are for blending
    bot_blend = bot_half[10:20]
    
    #linear blend the middle 10 frames
    bot_alpha = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.5, 0.6, 0.7, 0.8, 0.9]) #10 frames
    top_alpha = 1- bot_alpha
    blend = (bot_alpha[:, None, None]*bot_blend + top_alpha[:, None, None]*top_blend).astype(np.uint8) #linear blending
    
    #assemble
    full_seq[:nf_top-20] = top_use
    full_seq[nf_top-20:nf_top-10] = blend
    full_seq[nf_top-10:] = bot_use    
#     full_seq[:nf_top-10] = top_half
#     full_seq[nf_top:nf_top+nf_mid] = mid
#     full_seq[nf_top+nf_mid:nf_top+nf_mid+nf_bot] = bot_half
    image_show(full_seq[len(full_seq)//2])
    
    full_seq_dir = os.path.join(full_seq_folder, seq_name+'.tif')
    imsave(full_seq_dir, full_seq)


Z:\WX\WX_test_dod\whobio_sets\vid2vid_results\s001_o\test_latest\dod*
21


  0%|                                                                                           | 0/21 [00:00<?, ?it/s]

----------- dod_s001c_blk000
reading top half...


In [4]:
# create folders for each biopsy and move blocks into the folder
from shutil import move

biopsy_name_list = []
for seq_name in seq_name_list:
    cur_biopsy = seq_name[:9]
    biopsy_dir = os.path.join(full_seq_folder, cur_biopsy)
    if cur_biopsy not in biopsy_name_list:
        biopsy_name_list.append(cur_biopsy)
        os.makedirs(biopsy_dir)
    seq_dir = os.path.join(full_seq_folder,seq_name+'.tif')
    dest_dir = os.path.join(biopsy_dir, seq_name+'.tif')
    move(seq_dir, dest_dir)
print(len(biopsy_name_list))

1


In [5]:
# Calling imageJ macro for stitching blocks into full-res biopsy and save tiff image
# relying to fiji macro C:\\Users\\Administrator\\Documents\\fiji-win64\\Fiji.app\\macros\\stitch_biopsies_pythoncall.ijm
import subprocess

executable = "C:\\Users\\Administrator\\Documents\\fiji-win64\\Fiji.app\\ImageJ-win64.exe"

process_name = " -macro C:\\Users\\Administrator\\Documents\\fiji-win64\\Fiji.app\\macros\\stitch_biopsies_pythoncall.ijm "
# the blanks in process_name is important for the format to be read by fiji, don't change.

wait_buffer = subprocess.Popen(executable + process_name + biopsy_group, shell=True)

wait_buffer.wait() #wait for the stitching process to finish before finishing this cell

0